# Detección de Fraude en Tarjetas de Crédito utilizando SVM

Este notebook contiene la implementación de un modelo de **Support Vector Machine (SVM)** para detectar fraudes en transacciones con tarjetas de crédito.

## Pasos

1. **Carga de datos**: Exploración inicial del conjunto de datos proporcionado.
2. **Limpieza y Normalización de características**: Normalización de los datos y manejo del desequilibrio de clases.
3. **Entrenamiento del modelo SVM**: Implementación del algoritmo SVM con pesos balanceados.
4. **Evaluación**: Análisis de las métricas de desempeño (precision, recall, F1-score) y visualización de los resultados.


In [ ]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('/mnt/data/creditcard.csv')
df.info()
df.head()

## 2. Limpieza y Normalización de Características

### Desafíos:
- **Clase desequilibrada**: Hay muchas más transacciones legítimas que fraudulentas.
- **Normalización de características**: Características como el monto deben ser normalizadas para evitar sesgos en el modelo.

### Pasos realizados:
1. **Normalización del monto (Amount)** utilizando `StandardScaler` para escalar los valores entre un rango comparable.
2. **Manejo del desequilibrio** mediante `class_weight='balanced'` en el modelo SVM.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separar características y variable objetivo
X = df.drop(columns=['Class'])
y = df['Class']

# Normalizar la característica 'Amount'
scaler = StandardScaler()
X['Amount'] = scaler.fit_transform(X[['Amount']])

# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## 3. Entrenamiento del modelo SVM

El modelo SVM se entrena utilizando un balanceo de clases (`class_weight='balanced'`) para manejar el desequilibrio de las clases. Esto asegura que el modelo preste mayor atención a las transacciones fraudulentas durante el entrenamiento.

In [ ]:
from sklearn.svm import SVC

# Entrenar el modelo SVM con balanceo de clases
svm_model = SVC(kernel='rbf', class_weight='balanced', random_state=42)
svm_model.fit(X_train, y_train)

## 4. Evaluación del Modelo

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Hacer predicciones
y_pred = svm_model.predict(X_test)

# Reporte de clasificación y matriz de confusión
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## 5. Visualización de Resultados

Se mostrarán visualizaciones que incluyen la matriz de confusión y las métricas de precisión, recall y F1-score.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Graficar la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=['Legítimo', 'Fraude'], yticklabels=['Legítimo', 'Fraude'])
plt.ylabel('Actual')
plt.xlabel('Predicho')
plt.title('Matriz de Confusión del Modelo SVM')
plt.show()

In [ ]:
# Gráfico de barras para las métricas del modelo
report = classification_report(y_test, y_pred, output_dict=True)
metrics_df = pd.DataFrame(report).transpose()
metrics_df[['precision', 'recall', 'f1-score']].iloc[:2].plot(kind='bar', figsize=(8, 5))
plt.title('Métricas de Desempeño del Modelo SVM (Clase Legítima y Fraudulenta)')
plt.ylabel('Valor')
plt.xticks(rotation=0)
plt.show()